In [1]:
from notebook.services.config import ConfigManager
cm = ConfigManager()

#cm.update('livereveal', {'width': 1000, 'height': 600,'scroll': True,})
cm.update('livereveal', {'scroll': True,})

%load_ext autoreload
%autoreload 2

import os
import sys
sys.path.append(os.path.abspath("."))

from viewer import MeshCatViewer


In [2]:
%%html
<style>
body.rise-enabled div.code_cell{
    font-size:60%;
}

body.rise-enabled div.inner_cell>div.promt_container{
    width:10%;
}

body.rise-enabled div.inner_cell>div.text_cell_render.rendered_html {
    font-size: 50%;
}
</style>

# Robotic fabrication fundamentals

## Frame

A frame is defined by a base point and two orthonormal base vectors (xaxis, yaxis), which specify the normal (zaxis). 
It describes location and orientation in a (right-handed) cartesian coordinate system.

<img src='images/frame.svg' style='height:300px' />

In [3]:
"""There are several ways to construct a `Frame`.
"""
from compas.geometry import Point
from compas.geometry import Vector
from compas.geometry import Frame
from compas.geometry import Plane

F = Frame(Point(1, 0, 0), Vector(-0.45, 0.1, 0.3), Vector(1, 0, 0))
F = Frame([1, 0, 0], [-0.45, 0.1, 0.3], [1, 0, 0])
F = Frame.from_points([1, 1, 1], [2, 3, 6], [6, 3, 0])
F = Frame.from_plane(Plane([0, 0, 0], [0.5, 0.2, 0.1]))
F = Frame.from_euler_angles([0.5, 1., 0.2])
F = Frame.worldXY()
print(F)

Frame(Point(0.000, 0.000, 0.000), Vector(1.000, 0.000, 0.000), Vector(0.000, 1.000, 0.000))


### Frame as a cartesian coordinate system

<img src='images/point_in_frame.svg' style='height:350px' />

In [6]:
"""Example: 'point in frame'
"""
point = Point(146.00, 150.00, 161.50)
xaxis = Vector(0.9767, 0.0010, -0.214)
yaxis = Vector(0.1002, 0.8818, 0.4609)

# coordinate system F
F = Frame(point, xaxis, yaxis)

# point in F (local coordinates)
P = Point(35., 35., 35.)

# point in global (world) coordinates
P_ = F.to_world_coords(P)
print(P_)

# check
P2 = F.to_local_coords(P_)
print(P2)

Point(190.314, 164.389, 200.285)
Point(35.000, 35.000, 35.000)


The simple example above shows how to use a frame as a coordinate system: Starting from a point `P` in the local (user-defined, relative) coordinate system of frame `F`, i.e. its position is relative to the origin and orientation of `F`, we want to get the position `P_` of `P` in the global (world, absolute) coordinate system.

### Frame in frame

<img src='images/frame_in_frame_point.svg' style='height:450px' />

In [8]:
"""Example: 'frame in frame'
"""
# coordinate system F
point = Point(146.00, 150.00, 161.50)
xaxis = Vector(0.9767, 0.0010, -0.214)
yaxis = Vector(0.1002, 0.8818, 0.4609)
F = Frame(point, xaxis, yaxis)

# frame f in F (local coordinates)
point = Point(35., 35., 35.)
xaxis = Vector(0.604, 0.430, 0.671)
yaxis = Vector(-0.631, 0.772, 0.074)
f = Frame(point, xaxis, yaxis)

# frame in global (world) coordinate system
f_wcf = F.to_world_coords(f)
print(f_wcf)

# check
f2 = F.to_local_coords(f_wcf)
print(f2)
print(f == f2)

Frame(Point(190.314, 164.389, 200.285), Vector(0.760, 0.063, 0.647), Vector(-0.526, 0.645, 0.554))
Frame(Point(35.000, 35.000, 35.000), Vector(0.604, 0.430, 0.671), Vector(-0.631, 0.772, 0.074))
True


### Example

Bring a box from the world coordinate system into another coordinate system.

In [10]:
"""Example: Bring a box from the world coordinate system into another coordinate system.
"""
from compas.geometry import Frame
from compas.geometry import Box

# Box in the world coordinate system
frame = Frame([1, 0, 0], [-0.45, 0.1, 0.3], [1, 0, 0])
width, length, height = 1, 1, 1
box = Box(frame, width, length, height)

# Frame F representing a coordinate system
F = Frame([2, 2, 2], [0.978, 0.010, -0.210], [0.090, 0.882, 0.463])

# Represent box frame in frame F and construct new box
box_frame_transformed = F.to_world_coords(box.frame)
box_transformed = Box(box_frame_transformed, width, length, height)
print("Box frame transformed:", box_transformed.frame)

Box frame transformed: Frame(Point(2.978, 2.010, 1.790), Vector(-0.680, -0.105, 0.726), Vector(0.733, -0.132, 0.668))


In [23]:
viewer = MeshCatViewer()
viewer.draw_frame(Frame.worldXY())
viewer.draw_box(box)
viewer.draw_frame(F)
viewer.draw_box(box_transformed, color=0xff00ff)
viewer.jupyter_cell()

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7002/static/


In [45]:
from threejs_viewer import ThreeJsViewer
import pythreejs as p3js
viewer = ThreeJsViewer()
viewer.draw_frame(Frame.worldXY(), line_width=2)
viewer.draw_box(box)
viewer.draw_frame(F, line_width=2)
viewer.draw_box(box_transformed, color='#ff00ff')
viewer.show()

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(intensity=0.5, position=(0.0, 0.0, 1.0), quaterni…

### Further information

* https://en.wikipedia.org/wiki/Frame_of_reference
* https://en.wikipedia.org/wiki/Cartesian_coordinate_system

## Transformation

Transformations refer to operations such as moving, rotating, and scaling objects. They are stored using 4x4 transformation matrices. 

Most transformations preserve the parallel relationship among the parts of the geometry. For example collinear points 
remain collinear after the transformation. Also points on one plane stay coplanar after transformation. This type of
transformation is called an *affine transformation* and concerns transformations such as `Rotation`, `Translation`, 
`Scale`, `Reflection`, `Shear` and orthogonal and parallel `Projection`. Only perspective `Projection` is not *affine*.

<img src='images/transformations2D.svg' style='width:800px' />

A transformation matrix looks like this:

\begin{equation*}
\mathbf{T} = \begin{bmatrix}
a_{11} & a_{12} & a_{13} & a_{14}\\
a_{21} & a_{22} & a_{23} & a_{24}\\
a_{31} & a_{32} & a_{33} & a_{34}\\
a_{41} & a_{42} & a_{43} & a_{44}\\
\end{bmatrix}
\end{equation*}

But for different transformations, different coefficients are used:

The `Rotation` just uses the upper left 3x3 coefficients,

\begin{equation*}
\mathbf{T} = \begin{bmatrix}
a_{11} & a_{12} & a_{13} & 0\\
a_{21} & a_{22} & a_{23} & 0\\
a_{31} & a_{32} & a_{33} & 0\\
0 & 0 & 0 & 1\\
\end{bmatrix}
\end{equation*}

the `Translation` just uses the first 3 coefficients of the 4th column,

\begin{equation*}
\mathbf{T} = \begin{bmatrix}
1 & 0 & 0 & a_{14}\\
0 & 1 & 0 & a_{24}\\
0 & 0 & 1 & a_{34}\\
0 & 0 & 0 & 1\\
\end{bmatrix}
\end{equation*}

and `Scale` uses only the first 3 values of the diagonal.

\begin{equation*}
\mathbf{T} = \begin{bmatrix}
a_{11} & 0 & 0 & 0\\
0 & a_{22} & 0 & 0\\
0 & 0 & a_{33} & 0\\
0 & 0 & 0 & 1\\
\end{bmatrix}
\end{equation*}

In [ ]:
from compas.geometry import *

axis, angle = [0.2, 0.4, 0.1], 0.3
R = Rotation.from_axis_and_angle(axis, angle)
print("Rotation:\n", R)

translation_vector = [5, 3, 1]
T = Translation(translation_vector)
print("Translation:\n", T)

scale_factors = [0.1, 0.3, 0.4]
S = Scale(scale_factors)
print("Scale:\n", S)

point, normal = [0.3, 0.2, 1], [0.3, 0.1, 1]
R = Reflection(point, normal) # TODO: should take a plane!!
print("Reflection:\n", R)

point, normal = [0, 0, 0], [0, 0, 1]
perspective = [1, 1, 0]
P = Projection.perspective(point, normal, perspective)
print("Perspective projection:\n", R)

angle, direction = 0.1, [0.1, 0.2, 0.3]
point, normal = [4, 3, 1], [-0.11, 0.31, -0.17]
S = Shear(angle, direction, point, normal)
print("Shear:\n", S)


### Inverse transformation

The inverse transformation $T^{-1}$ is calculated through inverting the transformation matrix $T$. 

$T \times T^{-1} = I$, with $I$ as identity matrix. 

In [ ]:
"""Example: Transform a point and invert the transformation
"""
from compas.geometry import *
from math import pi

p = Point(3, 4, 5)
T = Rotation.from_axis_and_angle([2, 2, 2], pi/4)

#p.transform(T)
#print(p) # transformed point
#Tinv = T.inverse()
#p.transform(Tinv) # invert transformation
#print(p) # == (3, 4, 5)
#
#print(T * Tinv)

### Concatenation and decomposition

The concatenation of several `Transformations` are simple matrix multiplications.
But matrices are not commutative ($\mathbf{A}\times\mathbf{B} \neq \mathbf{B}\times\mathbf{A}$), 
so it is important to consider the order of multiplication.

If you transform an object first with transformation $\mathbf{A}$, then with transformation $\mathbf{B}$, 
followed by transformation $\mathbf{C}$, you get the same result as transforming the object with only 
one transformation $\mathbf{M}$ which is calculated by $\mathbf{M}=\mathbf{A}\times\mathbf{B}\times\mathbf{C}$.
(Transformations applied from left to right.)

**NOTE**:
<p style="background-color: yellow;">
If you want to apply several transformations on a big mesh, it is faster to multiply transformations first 
and then apply only one transformation to mesh.
</p>

In [ ]:
"""Example: multiply transformations
"""
from compas.geometry import *

translation = [1, 2, 3]
# A = Translation(translation)
axis, angle = [-0.8, 0.35, 0.5], 2.2
# B = Rotation.from_axis_and_angle(axis, angle)
scale_factors = [0.1, 0.3, 0.4]
# C = Scale(scale_factors)

# Concatenate transformations: apply transformations from left to right
# M = A * B * C

# A matrix can also be decomposed into it's components of Scale, 
# Shear, Rotation, Translation and Perspective
Sc, Sh, R, T, P = M.decompose()
# Check
print(C == Sc)
print(B == R)
print(A == T)

### Homogenisation

To transform a point, you simply multiply the point with the transformation matrix. But a transformation matrix is 4x4 
and vectors and points are 3x1, so how can they be multiplied?

To transform points and vectors, i.e. multiply them with the transformation matrix, we need to homogenize them first.
This means representing a 3-vector (x, y, z) as a 4-vector (x, y, z, 1) for points, or (x, y, z, 0) for vectors.

Points:
\begin{equation*}
\begin{bmatrix}x'\\y'\\z'\\1\end{bmatrix} = 
\begin{bmatrix}
a_{11} & a_{12} & a_{13} & a_{14}\\
a_{21} & a_{22} & a_{23} & a_{24}\\
a_{31} & a_{32} & a_{33} & a_{34}\\
a_{41} & a_{42} & a_{43} & a_{44}\\
\end{bmatrix}
\times
\begin{bmatrix}x\\y\\z\\1\end{bmatrix}
\end{equation*}

Vectors:
\begin{equation*}
\begin{bmatrix}x'\\y'\\z'\\0\end{bmatrix} = 
\begin{bmatrix}
a_{11} & a_{12} & a_{13} & a_{14}\\
a_{21} & a_{22} & a_{23} & a_{24}\\
a_{31} & a_{32} & a_{33} & a_{34}\\
a_{41} & a_{42} & a_{43} & a_{44}\\
\end{bmatrix}
\times
\begin{bmatrix}x\\y\\z\\0\end{bmatrix}
\end{equation*}

**NOTE**:
<p style="background-color: yellow;">
That is the reason for distinguishing between points and vectors!
</p>

In [ ]:
"""Example: transform point and vector
"""
frame = Frame([1, 0, 0], [-0.45, 0.1, 0.3], [1, 0, 0])
T = Transformation.from_frame(frame)
p = Point(1, 1, 1)
p.transform(T)
v = Vector(1, 1, 1)
v.transform(T)
print(p)
print(v)

## Transformation of multiple points

If you need to transform multiple points it is better to do this with the methods `transform_points` or `transform_points_numpy`
than applying the transformation on each point. (likewise `transform_vectors` or `transform_vectors_numpy`)

In [ ]:
import time
import compas
from compas.datastructures import Mesh
from compas.geometry import transform_points
from compas.geometry import transform_points_numpy

mesh = Mesh.from_ply(compas.get('bunny.ply'))
v, f = mesh.to_vertices_and_faces()
print("The mesh has {} vertices.".format(len(v)))
frame = Frame([1, 0, 0], [-0.45, 0.1, 0.3], [1, 0, 0])
T = Transformation.from_frame(frame)

# transform points with transform_points
t0 = time.time()
transform_points(v, T)
print("transfrom_points takes {:.4f} seconds.".format(time.time() - t0))

# transform points with transform_points_numpy
t0 = time.time()
transform_points_numpy(v, T)
print("transfrom_points_numpy takes {:.4f} seconds.".format(time.time() - t0))

## Frame and Transformation

<img src='images/frame_transformation.svg' style='height:450px' />

### Difference between change-basis transformation and transformation between frames.

`Transformation.change_basis(f1, f2)` and `Transformation.from_frame_to_frame(f1, f2)`

A change-basis transformation allows to remap geometry of one coordinate system in another, i.e. represent the same coordinates in 2 different frames,
whereas the Transformation between 2 frames allows to transform geometry from one coordinate system into the other one.

In [ ]:
"""This example computes a transformation between two frames F1 and F2.
"""
from compas.geometry import Frame
from compas.geometry import Transformation

F1 = Frame([2, 2, 2], [0.12, 0.58, 0.81], [-0.80, 0.53, -0.26])
F2 = Frame([1, 1, 1], [0.68, 0.68, 0.27], [-0.67, 0.73, -0.15])

T = Transformation.from_frame_to_frame(F1, F2)
print(T)

# now transform F1 with T
F2 == F1.transformed(T)

In [ ]:
"""This example computes a change-basis transformation between two frames F1 and F2.
"""
from compas.geometry import Frame
from compas.geometry import Transformation

F1 = Frame([2, 2, 2], [0.12, 0.58, 0.81], [-0.80, 0.53, -0.26])
F2 = Frame([1, 1, 1], [0.68, 0.68, 0.27], [-0.67, 0.73, -0.15])

T = Transformation.change_basis(F1, F2)
print(T)

# F1 local = worldXY F1 in F2
Point(0, 0, 0).transformed(T) == F2.local_coords(F1.point)

In [ ]:
"""Example: Bring a box from the world coordinate system into another coordinate system.
"""
from compas.geometry import Frame
from compas.geometry import Transformation
from compas.geometry import Box

# Box in the world coordinate system
frame = Frame([1, 0, 0], [-0.45, 0.1, 0.3], [1, 0, 0])
width, length, height = 1, 1, 1
box = Box(frame, width, length, height)

# Frame F representing a coordinate system
F = Frame([2, 2, 2], [0.978, 0.010, -0.210], [0.090, 0.882, 0.463])

# Get transformation between frames and apply transformation on box.
T = Transformation.from_frame_to_frame(Frame.worldXY(), F)
box_transformed = box.transformed(T)
print("Box frame transformed", box_transformed.frame)

In [ ]:
viewer = MeshCatViewer()
viewer.draw_frame(Frame.worldXY())
viewer.draw_box(box)
viewer.draw_frame(F)
viewer.draw_box(box_transformed, color=0x00aaff)
viewer.jupyter_cell()

<img src="https://gramaziokohler.github.io/compas_fab/latest/_images/02_coordinate_frames.jpg"  />

### Coordinate frames for robotic fabrication

Frames and representing coordinates in different coordinate frames (systems) is essential for planning robotic fabrication processes. 
For example, these frames are 

* World coordinate frame (WCF)

* Robot coordinate frame (RCF)

* Tool0 coordinate frame (T0CF)

* Tool coordinate frame (TCF)

* Object coordinate frame (OCF)

### Difference between `transform()` and `transformed()`


`T = Transformation.from_frame_to_frame(Frame.worldXY(), F)`


* Make a copy of box and transform it (don't change box):

    `box_transformed = box.transformed(T)`
                                       

* Transform box (returns `None`):

    `box.transform(T)`

## Rotation and orientation

A `Rotation` is a circular movement of an object around a point of rotation. A three-dimensional object can always be rotated around an infinite number of imaginary lines called rotation axes.


In [ ]:
"""There are several ways to construct a `Rotation`.
"""
import math
from compas.geometry import Rotation

R = Rotation.from_axis_and_angle([1, 0, 0], math.radians(30))
R = Rotation.from_axis_and_angle([1, 0, 0], math.radians(30), point=[1, 0, 0])
R = Rotation.from_basis_vectors([0.68, 0.68, 0.27], [-0.67, 0.73, -0.15])
R = Rotation.from_frame(Frame([1, 1, 1], [0.68, 0.68, 0.27], [-0.67, 0.73, -0.15]))
R = Rotation.from_axis_angle_vector([-0.043, -0.254, 0.617])
R = Rotation.from_quaternion([0.945, -0.021, -0.125, 0.303])
R = Rotation.from_euler_angles([1.4, 0.5, 2.3], static=True, axes='xyz')

print(R)

In [ ]:
from meshcat import Visualizer
from meshcat.animation import Animation
import meshcat.transformations as tf
import numpy as np
from compas.geometry import *
viewer = MeshCatViewer()

viewer.draw_sphere(Sphere((0,2,0), 0.5), id="sphere")

R1 = Rotation.from_axis_and_angle([0,0,1], math.radians(0), point=[0,0,0])
R2 = Rotation.from_axis_and_angle([0,0,1], math.radians(180), point=[0,0,0])
R3 = Rotation.from_axis_and_angle([0,0,1], math.radians(360), point=[0,0,0])

viewer.jupyter_cell()

In [ ]:
import numpy as np

animation = Animation()
with animation.at_frame(viewer, 0) as frame_vis:
    frame_vis.set_transform(np.array(R1.matrix))
with animation.at_frame(viewer, 100) as frame_vis:
    frame_vis.set_transform(np.array(R2.matrix))
with animation.at_frame(viewer, 200) as frame_vis:
    frame_vis.set_transform(np.array(R3.matrix))

viewer.set_animation(animation, repetitions=100)

### Euler angles

The Euler angles are three angles introduced by Leonhard Euler to describe the orientation of a rigid body with respect
to a fixed coordinate system.

The three elemental rotations may be 
* extrinsic (`static=True`, rotations about the axes xyz of the original coordinate system, which is assumed to remain motionless), or
* intrinsic (`static=False`, rotations about the axes of the rotating coordinate system XYZ, solidary with the moving body, which changes its orientation after each elemental rotation).

Euler angles are typically denoted as α, β, γ. Different authors use different sets of rotation axes to 
define Euler angles (`axes='xyz'`,`axes='xyx'`, ...) or different names for the same angles.
In flight dynamics, the principal rotations described with Euler angles are known as *pitch*, *roll* and *yaw*.

<img src='https://upload.wikimedia.org/wikipedia/commons/thumb/8/82/Euler.png/543px-Euler.png' style='height:450px' />

In [ ]:
from compas.geometry import *

box = Box(Frame([0,0,0], [1,0,0], [0,1,0]), 1., 1., 1.)
 
alpha, beta, gamma = 0.3, 0.5, 0.7
xaxis, yaxis, zaxis = [1, 0, 0], [0, 1, 0], [0, 0, 1]

Rx = Rotation.from_axis_and_angle(xaxis, alpha)
Ry = Rotation.from_axis_and_angle(yaxis, beta)
Rz = Rotation.from_axis_and_angle(zaxis, gamma)

In [ ]:
from meshcat import Visualizer
viewer = MeshCatViewer()
viewer.draw_box(box, id="box")
viewer.draw_frame(box.frame, id="f")
viewer.jupyter_cell()

In [ ]:
from meshcat.animation import Animation
import meshcat.transformations as tf
import numpy as np

R2 = Rx * Ry
R3 = Rx * Ry * Rz
obj_ids = ['f_xaxis', 'f_yaxis', 'f_zaxis', 'box']

animation = Animation()
with animation.at_frame(viewer, 0) as frame_vis:
    for id in obj_ids:
        frame_vis[id].set_transform(np.array(Transformation().matrix))
with animation.at_frame(viewer, 100) as frame_vis:
    for id in obj_ids:
        frame_vis[id].set_transform(np.array(Rx.matrix))
# pause
with animation.at_frame(viewer, 130) as frame_vis:
    for id in obj_ids:
        frame_vis[id].set_transform(np.array(Rx.matrix))
with animation.at_frame(viewer, 230) as frame_vis:
    for id in obj_ids:
        frame_vis[id].set_transform(np.array(R2.matrix))
# pause
with animation.at_frame(viewer, 260) as frame_vis:
    for id in obj_ids:
        frame_vis[id].set_transform(np.array(R2.matrix))
with animation.at_frame(viewer, 360) as frame_vis:
    for id in obj_ids:
        frame_vis[id].set_transform(np.array(R3.matrix))
    
viewer.set_animation(animation)

In [ ]:
"""Example: Rotations from euler angles, rotate an object based on 3 euler angles.
"""

import compas
from compas.geometry import Frame
from compas.geometry import Rotation
from compas.datastructures import Mesh
from compas.datastructures import mesh_transform

# euler angles
alpha, beta, gamma = -0.156, -0.274, 0.785
static, axes = True, 'xyz'

# Version 1: Create Rotation from angles
R1 = Rotation.from_euler_angles([alpha, beta, gamma], static, axes)

# Version 2: Concatenate 3 Rotations
xaxis, yaxis, zaxis = [1, 0, 0], [0, 1, 0], [0, 0, 1]
Rx = Rotation.from_axis_and_angle(xaxis, alpha)
Ry = Rotation.from_axis_and_angle(yaxis, beta)
Rz = Rotation.from_axis_and_angle(zaxis, gamma)
 
if static: # check order!
    R2 = Rz * Ry * Rx
else:
    R2 = Rx * Ry * Rz

# Check
print(R1 == R2)

### Axis-angle representation

The axis–angle representation of a rotation parameterizes a rotation in a three-dimensional Euclidean space by two 
quantities: a unit vector $\mathbf{e}$ indicating the direction of an axis of rotation, and an angle $θ$ describing 
the magnitude of the rotation about the axis.

<img src='https://upload.wikimedia.org/wikipedia/commons/thumb/7/7b/Angle_axis_vector.svg/300px-Angle_axis_vector.svg.png' style='height:250px' />

In [ ]:
"""Example: Create a rotation from and axis and an angle.
"""
from compas.geometry import *
aav = Vector(-0.043, -0.254, 0.617)
angle, axis = aav.unitized(), aav.length
print(angle, axis)
R = Rotation.from_axis_angle_vector(aav)
axis, angle = R.axis_and_angle
print(axis, angle)

### Quaternion

In mathematics, the quaternions are a number system that extends the complex numbers. 

Quaternions are generally represented in the form:

${\displaystyle a+b\ \mathbf{i} +c\ \mathbf{j} +d\ \mathbf{k} }$

where $a$, $b$, $c$, and $d$ are real numbers, and $\mathbf{i}$, $\mathbf{j}$, and $\mathbf{k}$ are the fundamental quaternion units.

Unit quaternions, also known as versors, provide a convenient mathematical notation for representing orientations and 
rotations of objects in three dimensions. Compared to Euler angles they are simpler to compose and compared to 
rotation matrices they are more compact, more numerically stable, and more efficient. 


In [ ]:
from compas.geometry import Rotation
from compas.geometry import Quaternion

q = Quaternion(0.918958, -0.020197, -0.151477, 0.363544)
print(q.is_unit)
R = Rotation.from_quaternion(q)
print(R.quaternion == q)


In [ ]:
"""Example: Different Robot vendors use different conventions to describe TCP orientation."""

from compas.geometry import Point
from compas.geometry import Vector
from compas.geometry import Frame
point = Point(0.0, 0.0, 63.0)
xaxis = Vector(0.68, 0.68, 0.27)
yaxis = Vector(-0.67, 0.73, -0.15)
F = Frame(point, xaxis, yaxis)
print(F.quaternion) # ABB
print(F.euler_angles(static=False, axes='xyz')) # Staubli
print(F.euler_angles(static=False, axes='zyx')) # KUKA
print(F.axis_angle_vector) # UR

In [ ]:
"""Example: Project the points of a box onto the xy-plane.
"""

from compas.geometry import Frame
from compas.geometry import Projection
from compas.geometry import transform_points
from compas.geometry import Box

from compas.datastructures import Mesh

# create a box
frame = Frame([0, 0, 2], [-0.47, 0.85, -0.24], [-0.75, -0.53, -0.39])
box = Box(frame, 1, 1, 1)

# try different projections
point, normal = [0, 0, 0], [0, 0, 1] # projection plane is xy plane
# project orthogonal onto the plane
P = Projection.orthogonal(point, normal)
# project parallel with a certain direction
P = Projection.parallel(point, normal, [-0.5, 0.1, -0.7])
# project perspective from a certain view point
P = Projection.perspective(point, normal, [0.6, -1, 6])

# Perform the transformation onto the box' vertices 
pts = transform_points(box.vertices, P)
    
viewer = MeshCatViewer()
viewer.draw_box(box)
# draw edges between the projected points
mesh = Mesh.from_vertices_and_faces(pts, box.faces)
viewer.draw_mesh_edges(mesh, color=0xffff00)
viewer.jupyter_cell()

### Further information

* https://en.wikipedia.org/wiki/Transformation_matrix
* https://en.wikipedia.org/wiki/Euler_angles
* https://en.wikipedia.org/wiki/Axis%E2%80%93angle_representation
* https://en.wikipedia.org/wiki/Quaternions_and_spatial_rotation